## `HURDAT2` Data Munge

This notebook acquires, cleans up, and saves a copy of the Atlantic segment of the United States National Oceanic and Atmospheric Administration's (NOAA) HURDAT2 dataset.

In [1]:
import requests
atlantic_raw = requests.get(
    "https://www.nhc.noaa.gov/data/hurdat/hurdat2-1851-2017-050118.txt"
)
atlantic_raw.raise_for_status()  # check that we actually got something back

Because of a non-standard format, a naive `pandas.read_csv` won't get useable data. It will be confused about the storm subheadings, for example, the first row in the following block:

```
EP202015,           PATRICIA,     19,
20151020, 0600,  , TD, 13.4N,  94.0W,  25, 1007,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
20151020, 1200,  , TD, 13.3N,  94.2W,  30, 1006,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
20151020, 1800,  , TD, 13.2N,  94.6W,  30, 1006,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
...
```

The code that follows builds a parser that reads the data into a `pandas` `DataFrame` line by line.

In [2]:
import io
from collections import Counter

c = Counter()
for line in io.StringIO(atlantic_raw.text).readlines():
    c[line[:2]] += 1

atlantic_storms_r = []
atlantic_storm_r = {'header': None, 'data': []}

for i, line in enumerate(io.StringIO(atlantic_raw.text).readlines()):
    if line[:2] == 'AL':
        atlantic_storms_r.append(atlantic_storm_r.copy())
        atlantic_storm_r['header'] = line
        atlantic_storm_r['data'] = []
    else:
        atlantic_storm_r['data'].append(line)

atlantic_storms_r = atlantic_storms_r[1:]

import pandas as pd

atlantic_storm_dfs = []
for storm_dict in atlantic_storms_r:
    storm_id, storm_name, storm_entries_n = storm_dict['header'].split(",")[:3]
    data = [[entry.strip() for entry in datum[:-1].split(",")] for datum in storm_dict['data']]
    frame = pd.DataFrame(data)
    frame['id'] = storm_id
    frame['name'] = storm_name
    atlantic_storm_dfs.append(frame)
    
atlantic_storms = pd.concat(atlantic_storm_dfs)
atlantic_storms = atlantic_storms.reindex(columns=atlantic_storms.columns[-2:] | atlantic_storms.columns[:-2])

In [3]:
import numpy as np

# Assign columns from the metadata.
atlantic_storms.columns = [
        "id",
        "name",
        "date",
        "hours_minutes",
        "record_identifier",
        "status_of_system",
        "latitude",
        "longitude",
        "maximum_sustained_wind_knots",
        "maximum_pressure",
        "34_kt_ne",
        "34_kt_se",
        "34_kt_sw",
        "34_kt_nw",
        "50_kt_ne",
        "50_kt_se",
        "50_kt_sw",
        "50_kt_nw",
        "64_kt_ne",
        "64_kt_se",
        "64_kt_sw",
        "64_kt_nw",
        "na"
]

# Replace sentinal values with true NAs.
del atlantic_storms['na']
atlantic_storms = atlantic_storms.replace(to_replace='-999', value=np.nan)
atlantic_storms = atlantic_storms.replace(to_replace="", value=np.nan)

In [4]:
# Fix date and location columns.
atlantic_storms['latitude'] = atlantic_storms['latitude']\
    .map(lambda lat: lat[:-1] if lat[-1] == "N" else -lat[:-1])
atlantic_storms['longitude']= atlantic_storms['longitude']\
    .map(lambda long: long[:-1] if long[-1] == "E" else "-" + long[:-1])
atlantic_storms['date'] = pd.to_datetime(atlantic_storms['date'])
atlantic_storms['date'] = atlantic_storms\
    .apply(
        lambda srs: srs['date'].replace(hour=int(srs['hours_minutes'][:2]), minute=int(srs['hours_minutes'][2:])), 
        axis='columns'
    )

# Remove unused column.
del atlantic_storms['hours_minutes']

# Strip out spaces padding out names.
atlantic_storms['name'] = atlantic_storms['name'].map(lambda n: n.strip())

# Reindex.
atlantic_storms.index = range(len(atlantic_storms.index))
atlantic_storms.index.name = "index"

In [5]:
pd.set_option("max_columns", None)
atlantic_storms.head()

,id,name,date,record_identifier,status_of_system,latitude,longitude,maximum_sustained_wind_knots,maximum_pressure,34_kt_ne,34_kt_se,34_kt_sw,34_kt_nw,50_kt_ne,50_kt_se,50_kt_sw,50_kt_nw,64_kt_ne,64_kt_se,64_kt_sw,64_kt_nw
index,,,,,,,,,,,,,,,,,,,,,
0,AL011851,UNNAMED,1851-06-25 00:00:00,NaN,HU,28.0,-94.8,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL011851,UNNAMED,1851-06-25 06:00:00,NaN,HU,28.0,-95.4,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL011851,UNNAMED,1851-06-25 12:00:00,NaN,HU,28.0,-96.0,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AL011851,UNNAMED,1851-06-25 18:00:00,NaN,HU,28.1,-96.5,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AL011851,UNNAMED,1851-06-25 21:00:00,L,HU,28.2,-96.8,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
